In [36]:
## Import packages

import pandas as pd
import os
import numpy as np
import math
from scipy.stats import pearsonr
import requests
from datetime import date
from datetime import datetime


pd.set_option('display.max_columns', None)

In [37]:
date = '2019-JUN-21'

str(int(date[:4]) - 1)

'2018'

In [38]:
## Input date for projections
## Get slate and player information from DFS sites

today = date

response = requests.get(f'https://api.sportsdata.io/api/mlb/fantasy/json/DfsSlatesByDate/{today}', headers={'Ocp-Apim-Subscription-Key': '6fcab751d8594ce9909283dcdc522d24'})
games = response.json()
df_slates = pd.json_normalize(games)
df_slates = df_slates[['SlateID', 'Operator', 'OperatorSlateID', 'OperatorName', 'NumberOfGames', 'OperatorGameType', 'SalaryCap']]
df_player_sal = pd.json_normalize(games, record_path =['DfsSlatePlayers'])
df_player_sal = df_player_sal[(df_player_sal['OperatorPosition'] != 'SP') & (df_player_sal['OperatorPosition'] != 'RP') & (df_player_sal['OperatorPosition'] != 'P')].reset_index(drop=True)
df_player_sal = df_player_sal.merge(df_slates, how='left', on='SlateID')

df_player_sal.head()



,SlatePlayerID,SlateID,SlateGameID,PlayerID,PlayerGameProjectionStatID,OperatorPlayerID,OperatorSlatePlayerID,OperatorPlayerName,OperatorPosition,OperatorSalary,RemovedByOperator,Team,TeamID,OperatorRosterSlots,Operator,OperatorSlateID,OperatorName,NumberOfGames,OperatorGameType,SalaryCap
0,4255376,9508,58231.0,10000217,2712490.0,390610,12847954,Anthony Rizzo,1B,9800,False,CHC,9.0,[UTIL],DraftKings,27878,NYM vs CHC,1,Showdown,50000.0
1,4255377,9508,58231.0,10000217,2712490.0,390610,12848092,Anthony Rizzo,1B,14700,False,CHC,9.0,[CPT],DraftKings,27878,NYM vs CHC,1,Showdown,50000.0
2,4255378,9508,58231.0,10007009,2712470.0,873981,12847951,Pete Alonso,1B,10400,False,NYM,18.0,[UTIL],DraftKings,27878,NYM vs CHC,1,Showdown,50000.0
3,4255379,9508,58231.0,10007009,2712470.0,873981,12848089,Pete Alonso,1B,15600,False,NYM,18.0,[CPT],DraftKings,27878,NYM vs CHC,1,Showdown,50000.0
4,4255380,9508,58231.0,10002076,2712508.0,549381,12847958,Kris Bryant,3B/OF,9000,False,CHC,9.0,[UTIL],DraftKings,27878,NYM vs CHC,1,Showdown,50000.0


In [30]:
## SKIP
## Get current season stats
## For beta testing just need to use game logs rather than full season

season = '2019'
response = requests.get(f'https://api.sportsdata.io/api/mlb/fantasy/json/PlayerSeasonStats/{season}', headers={'Ocp-Apim-Subscription-Key': '6fcab751d8594ce9909283dcdc522d24'})

stats = response.json()
df_player_season_stats = pd.json_normalize(stats)
                        

,StatID,TeamID,PlayerID,SeasonType,Season,Name,Team,Position,PositionCategory,Started,AverageDraftPosition,Games,FantasyPoints,AtBats,Runs,Hits,Singles,Doubles,Triples,HomeRuns,RunsBattedIn,BattingAverage,Outs,Strikeouts,Walks,HitByPitch,Sacrifices,SacrificeFlies,GroundIntoDoublePlay,StolenBases,CaughtStealing,OnBasePercentage,SluggingPercentage,OnBasePlusSlugging,Wins,Losses,Saves,InningsPitchedDecimal,TotalOutsPitched,InningsPitchedFull,InningsPitchedOuts,EarnedRunAverage,PitchingHits,PitchingRuns,PitchingEarnedRuns,PitchingWalks,PitchingStrikeouts,PitchingHomeRuns,PitchesThrown,PitchesThrownStrikes,WalksHitsPerInningsPitched,PitchingBattingAverageAgainst,FantasyPointsFanDuel,FantasyPointsDraftKings,WeightedOnBasePercentage,PitchingCompleteGames,PitchingShutOuts,PitchingOnBasePercentage,PitchingSluggingPercentage,PitchingOnBasePlusSlugging,PitchingStrikeoutsPerNineInnings,PitchingWalksPerNineInnings,PitchingWeightedOnBasePercentage
0,2158239,32,10000001,1,2018,Chase Anderson,MIL,SP,P,30,None,31,136.00,45.0,1.0,4.0,4.0,0.0,0.0,0.0,2.0,0.089,41.0,23.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.089,0.089,0.178,10.0,9.0,0.0,158.000000,474.0,158.0,0.0,3.930,131.0,71.0,69.0,57.0,128.0,30.0,2602.0,1656.0,1.190,0.228,755.0,396.5,0.089,0.0,0.0,0.303,0.426,0.729,7.29,3.25,0.321
1,2206154,24,10000004,1,2018,Trevor Cahill,OAK,SP,P,20,None,21,92.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,7.0,4.0,0.0,110.000000,330.0,110.0,0.0,3.764,90.0,52.0,46.0,41.0,100.0,8.0,1791.0,1087.0,1.191,0.226,566.0,301.9,0.000,0.0,0.0,0.302,0.347,0.649,8.18,3.35,0.298
2,2158229,14,10000005,1,2018,Andrew Chafin,ARI,RP,P,0,None,77,36.33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,1.0,6.0,0.0,49.333333,148.0,49.0,1.0,3.101,41.0,18.0,17.0,25.0,53.0,0.0,878.0,533.0,1.338,0.225,262.0,146.2,0.000,0.0,0.0,0.321,0.297,0.618,9.67,4.56,0.288
3,2158185,14,10000007,1,2018,Patrick Corbin,ARI,SP,P,33,None,33,195.00,61.0,1.0,12.0,11.0,1.0,0.0,0.0,5.0,0.197,49.0,25.0,2.0,0.0,4.0,0.0,3.0,0.0,0.0,0.222,0.213,0.435,11.0,7.0,0.0,200.000000,600.0,200.0,0.0,3.150,162.0,70.0,70.0,48.0,246.0,15.0,3142.0,2020.0,1.050,0.218,1270.0,722.0,0.206,1.0,1.0,0.269,0.337,0.606,11.07,2.16,0.268
4,2378696,14,10000009,1,2018,Randall Delgado,ARI,RP,P,0,None,10,13.33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,2.0,0.0,0.0,11.333333,34.0,11.0,1.0,4.765,11.0,6.0,6.0,6.0,5.0,3.0,202.0,124.0,1.500,0.244,43.0,21.3,0.000,0.0,0.0,0.333,0.467,0.800,3.97,4.76,0.333


In [40]:
## Get game logs for in season performance calculations
## Can use the PlayerSeason files rather than PlayerGame once projections go live

os.chdir('..')
cwd = os.getcwd()
os.chdir(cwd + '/Fantasy.2018-2021')

## Currently using all past games we have access to for testing
## In practice you'd just need the current season data file
game_stats = pd.read_csv('PlayerGame.2019.csv')


/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
## Parse dataframe to only relevant data

# Get list of players on that day's slate
players = list(df_player_sal.PlayerID.unique())

# Select only regular season data (should only be needed on backtesting)
game_stats = game_stats.loc[game_stats.SeasonType == 1].reset_index(drop=True)

# Select only batter data
game_stats = game_stats.loc[game_stats.PositionCategory != 'P'].reset_index(drop=True)

# Calculate plate appearances by game
game_stats['PlateAppearances'] = game_stats.apply(lambda row: row['AtBats'] + row['Walks'] + row['HitByPitch'] + row['Sacrifices'], axis=1)

# Just changes name of dataframe
data = game_stats.sort_values(['PlayerID', 'Day'], ascending=True).reset_index(drop=True)

# For beta testing only, need to select only games from prior to the request date
data['Day'] = data['Day'].astype('datetime64[ns]')
data['Day'] = data['Day'].dt.date
date_object = datetime.strptime(today, '%Y-%b-%d').date()

data = data[data['Day'] < date_object].reset_index(drop=True)

data.rename(columns = {'Runs': 'R', 'Singles': 'S', 'Doubles': 'D', 'Triples': 'T', 'HomeRuns': 'HR', 'AtBats': 'AB', 'Walks':'BB', 'RunsBattedIn': 'RBI', 'PlateAppearances': 'PA', 'Hits': 'H', 'HitByPitch': 'HP', 'StolenBases': 'SB', 'CaughtStealing': 'CS', 'Strikeouts': 'SO'},  
          inplace = True) 

In [42]:
## This next step is necessary for creating projections for players without in season data or a pre season projection
## Average all stats by position

average_stats_by_position = data[['Position', 'PA', 'S', 'D', 'T', 'HR', 'BB', 'HP', 'SB', 'CS']].reset_index(drop=True).groupby(['Position']).sum()

## DH and PH stats get lumped into 1B position
average_stats_by_position.loc['1B'] = average_stats_by_position.loc[['1B', 'DH', 'PH']].sum()

## DFS sites only use the OF position, so LF, RF, and CF get lumped in together
average_stats_by_position.loc['OF'] = average_stats_by_position.loc[['LF', 'RF', 'CF']].sum()
average_stats_by_position = average_stats_by_position.drop(['DH', 'PH', 'PR', 'LF', 'RF', 'CF'])
average_stats_by_position



,PA,S,D,T,HR,BB,HP,SB,CS
Position,,,,,,,,,
1B,16291.0,2090.0,723.0,45.0,715.0,1671.0,166.0,85.0,54.0
2B,9525.0,1400.0,403.0,39.0,282.0,735.0,99.0,145.0,50.0
3B,9747.0,1397.0,451.0,33.0,356.0,866.0,107.0,86.0,31.0
C,8981.0,1236.0,379.0,14.0,309.0,760.0,128.0,29.0,17.0
SS,9535.0,1437.0,512.0,62.0,296.0,747.0,60.0,192.0,66.0
OF,29156.0,4083.0,1357.0,147.0,1074.0,2560.0,348.0,510.0,179.0


In [43]:
## Get player stats from prior season to establish a prior league average
## Players with no current data get a league average projection for their position
## The prior seasons league average gets a bit of weight during the current season

os.chdir('..')
cwd = os.getcwd()
os.chdir(cwd + '/Fantasy.2018-2021')
prior_season_stats = pd.read_csv('PlayerSeason.2018.csv')

In [44]:
## Clean up prior season data

prior_season_stats = prior_season_stats.loc[prior_season_stats.SeasonType == 1].reset_index(drop=True)
prior_season_stats = prior_season_stats.loc[prior_season_stats.PositionCategory != 'P'].reset_index(drop=True)
prior_season_stats['PlateAppearances'] = prior_season_stats.apply(lambda row: row['AtBats'] + row['Walks'] + row['HitByPitch'] + row['Sacrifices'], axis=1)
prior_season_stats.rename(columns = {'Runs': 'R', 'Singles': 'S', 'Doubles': 'D', 'Triples': 'T', 'HomeRuns': 'HR', 'AtBats': 'AB', 'Walks':'BB', 'RunsBattedIn': 'RBI', 'PlateAppearances': 'PA', 'Hits': 'H', 'HitByPitch': 'HP', 'StolenBases': 'SB', 'CaughtStealing': 'CS', 'Strikeouts': 'SO'},  
          inplace = True) 

In [45]:
## Find per plate appearance averages for each relevant position

prior_season_league_stats = prior_season_stats[['Position', 'PA', 'S', 'D', 'T', 'HR', 'BB', 'HP', 'SB', 'CS']].reset_index(drop=True).groupby(['Position']).sum()
prior_season_league_stats.loc['1B'] = prior_season_league_stats.loc[['1B', 'DH']].sum()
prior_season_league_stats.loc['OF'] = prior_season_league_stats.loc[['LF', 'RF', 'CF']].sum()
prior_season_league_stats = prior_season_league_stats.drop(['DH', 'LF', 'RF', 'CF'])
prior_season_league_stats = prior_season_league_stats.div(prior_season_league_stats.PA, axis=0)
prior_season_league_stats

,PA,S,D,T,HR,BB,HP,SB,CS
Position,,,,,,,,,
1B,1.0,0.136266,0.044876,0.002521,0.039581,0.099542,0.010337,0.004790,0.002185
2B,1.0,0.154518,0.045473,0.004770,0.027819,0.081316,0.009184,0.019972,0.006776
3B,1.0,0.140409,0.049667,0.003756,0.032908,0.086001,0.011220,0.006666,0.003756
C,1.0,0.136252,0.041338,0.002448,0.027020,0.080273,0.014734,0.003279,0.002032
SS,1.0,0.151723,0.047485,0.006152,0.025954,0.073355,0.008216,0.018455,0.007837
OF,1.0,0.144040,0.045107,0.005957,0.031092,0.089975,0.010564,0.018667,0.006618


In [46]:
## Find a league average to use in the current season

positions = ['1B', '2B', '3B', 'SS', 'C', 'OF']

for pos in positions:
    total = prior_season_league_stats.loc[pos] * 2000 + average_stats_by_position.loc[pos]
    new_row = total.divide(total.PA)
    average_stats_by_position.loc[pos] = new_row

In [47]:
## Select only data for players on the current days slates
## Sum all this data to find current season totals for each player

data = data.loc[data['PlayerID'].isin(players)]
sum_data = data[['PlayerID', 'PA', 'S', 'D', 'T', 'HR', 'BB', 'HP', 'SB', 'CS']].reset_index(drop=True).groupby(['PlayerID']).sum()
sum_data.head()

,PA,S,D,T,HR,BB,HP,SB,CS
PlayerID,,,,,,,,,
10000029,292.0,43.0,20.0,2.0,5.0,21.0,2.0,4.0,0.0
10000030,226.0,25.0,12.0,1.0,6.0,11.0,1.0,0.0,0.0
10000031,316.0,51.0,7.0,0.0,14.0,36.0,2.0,0.0,1.0
10000033,18.0,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0
10000034,145.0,11.0,4.0,1.0,2.0,8.0,2.0,4.0,1.0


In [48]:
## Get pre season projections file

os.chdir('..')
cwd = os.getcwd()
os.chdir(cwd + '/BaselineProjections')
marcels = pd.read_csv('marcel_batters_2019.csv')

In [49]:
## Add reliability metric to current season data

marcels['Name'] = marcels.apply(lambda row: row['First'] + ' ' + row['Last'], axis=1)
marcels.rename(columns = {'rel': 'Reliability'}, inplace = True)
df = marcels[['Name', 'Reliability']]
data = data.merge(df, how='left', on='Name')

In [50]:
## Create dictionary of reliability metric for ease of use

reliability_dict = {}

for index, row in data.iterrows():
    if math.isnan(row['Reliability']) == True:
        reliability_dict[row['PlayerID']] = 0
    else:
        reliability_dict[row['PlayerID']] = row['Reliability']

In [51]:
## Create dictionary of current season total stats and stabilization factors for each player

player_dict = {}
if sum_data.shape[0] == 0: pass # need to include something for the first day of the season
else:
    player_id_list = list(sum_data.index.values)
for player in player_id_list:
    player_dict[player] = {}
    for stat in ['PA', 'S', 'D', 'T', 'HR', 'BB', 'HP', 'SB', 'CS']:
        value = sum_data.loc[player, stat]
        player_dict[player][stat] = value

    PA = player_dict[player]['PA']
    BB = player_dict[player]['BB']
    HBP = player_dict[player]['HP']
    HR = player_dict[player]['HR']
    S = player_dict[player]['S']
    D = player_dict[player]['D']
    T = player_dict[player]['T']
    CS = player_dict[player]['CS']

    rel = reliability_dict[player]
    rel_fact = (2.2 ** rel) / 2

    player_dict[player]['BB_s'] = PA / (PA + (120 * rel_fact))
    player_dict[player]['HBP_s'] = PA / (PA + (240 * rel_fact))
    player_dict[player]['S_s'] = PA / (PA + (290 * rel_fact))
    player_dict[player]['D_s'] = (D + T) / ((D + T) + (48 * rel_fact))
    player_dict[player]['T_s'] = (D + T) / ((D + T) + (48 * rel_fact))
    player_dict[player]['HR_s'] = PA / (PA + (170 * rel_fact))
    player_dict[player]['SBA_s'] = (S + BB + HBP) / ((S + BB + HBP) + (39 * rel_fact))
    

In [52]:
## Convert full pre season projections to per plate appearance rates

data_ID = data[['PlayerID', 'Name']].drop_duplicates()
marcels = marcels.merge(data_ID, how='left', on='Name')
marcels = marcels[marcels['PlayerID'].notna()]
marcels = marcels.set_index('PlayerID')
rel_columns = marcels.columns.to_list()[12:29]
marcels = marcels[rel_columns]
marcels = marcels.div(marcels.PA, axis=0)
marcels['S'] = marcels.apply(lambda row: row['H'] - row['D'] - row['T'] - row['HR'], axis=1)
marcel_players = marcels.index.to_list()
marcels_dict = marcels.to_dict('index')

In [53]:
players_with_games = list(sum_data.index)
players_with_pre_season_projection = list(marcels.index)

In [54]:
## Find main position for each player

new = df_player_sal['OperatorPosition'].str.split('/', n = 1, expand = True)
df_player_sal['EffectivePosition'] = new[0]


In [55]:
## Create blended projections for the request date
## Different methods based on availability of pre season projections

player_projs_dict = {}

for player in players:
    
    if player in marcel_players: # if they have a marcels projection
    
        new_player_dict = {}
        stat_list = ['S', 'D', 'T', 'HR', 'BB', 'HP', 'SB']
        stab_list = ['S_s', 'D_s', 'T_s', 'HR_s', 'HBP_s', 'SBA_s']
        i = 0

        if player in player_dict:
            
            player_proj = player_dict[player]
            
        else: # If they do not have games (3)
            
            for stat in stat_list:
                new_player_dict[stat] = marcels_dict[player][stat]
                
            player_projs_dict[player] = new_player_dict
            continue

        # both games and marcel (1)

        PA = player_proj['PA']
        
        if PA == 0:
            
            for stat in stat_list:
                new_player_dict[stat] = marcels_dict[player][stat]
                
            player_projs_dict[player] = new_player_dict
            continue
            

        for stat in stat_list:

            if stat == 'SB':

                opps = player_proj['S'] + player_proj['HP'] + player_proj['BB']
                sba_exp = (marcels_dict[player][stat] + marcels_dict[player]['CS']) / (marcels_dict[player]['S'] + marcels_dict[player]['BB'] + marcels_dict[player]['HP'])
                if opps == 0:
                    sba_act = 0
                else:
                    sba_act = (player_proj[stat] + player_proj['CS']) / opps
                sba_blend = (sba_act * player_proj['SBA_s']) + (sba_exp * (1 - player_proj['SBA_s']))
                succ_rate_proj = marcels_dict[player][stat] / (marcels_dict[player][stat] + marcels_dict[player]['CS'])

            else:

                stat_exp = marcels_dict[player][stat] * PA
                stat_act = player_proj[stat]
                stat_blend = (stat_act * player_proj[stab_list[i]]) + (stat_exp * (1 - player_proj[stab_list[i]]))

                new_player_dict[stat_list[i]] = stat_blend / PA

                i += 1

        new_player_dict['SB'] = sba_blend * succ_rate_proj * (new_player_dict['S'] + new_player_dict['BB'] - new_player_dict['HP'])

        player_projs_dict[player] = new_player_dict
    
    else: 
        
        if player in player_dict: # Check if they've played games
            # No marcels, but games (2)
            # position average acts as default marcels projections
            player_proj = player_dict[player]
            
            eff_pos = df_player_sal.loc[df_player_sal['PlayerID'] == player, 'EffectivePosition'].iloc[0]
            proj_by_position = average_stats_by_position.loc[eff_pos]
            proj_by_pos_dict = proj_by_position.to_dict()
            
            new_player_dict = {}
            stat_list = ['S', 'D', 'T', 'HR', 'BB', 'HP', 'SB']
            stab_list = ['S_s', 'D_s', 'T_s', 'HR_s', 'HBP_s', 'SBA_s']
            i = 0
            
            PA = player_proj['PA']
        
            if PA == 0:

                for stat in stat_list:
                    new_player_dict[stat] = proj_by_pos_dict[player][stat]

                player_projs_dict[player] = new_player_dict
                continue


            for stat in stat_list:

                if stat == 'SB':

                    opps = player_proj['S'] + player_proj['HP'] + player_proj['BB']
                    sba_exp = (proj_by_pos_dict[stat] + proj_by_pos_dict['CS']) / (proj_by_pos_dict['S'] + proj_by_pos_dict['BB'] + proj_by_pos_dict['HP'])
                    if opps == 0:
                        sba_act = 0
                    else:
                        sba_act = (player_proj[stat] + player_proj['CS']) / opps
                    sba_blend = (sba_act * player_proj['SBA_s']) + (sba_exp * (1 - player_proj['SBA_s']))
                    succ_rate_proj = proj_by_pos_dict[stat] / (proj_by_pos_dict[stat] + proj_by_pos_dict['CS'])

                else:

                    stat_exp = proj_by_pos_dict[stat] * PA
                    stat_act = player_proj[stat]
                    stat_blend = (stat_act * player_proj[stab_list[i]]) + (stat_exp * (1 - player_proj[stab_list[i]]))

                    new_player_dict[stat_list[i]] = stat_blend / PA

                    i += 1

            new_player_dict['SB'] = sba_blend * succ_rate_proj * (new_player_dict['S'] + new_player_dict['BB'] - new_player_dict['HP'])

            player_projs_dict[player] = new_player_dict
            
        else: # Neither games nor marcels (4)
        
            # Find effective position

            eff_pos = df_player_sal.loc[df_player_sal['PlayerID'] == player, 'EffectivePosition'].iloc[0]
            proj_by_position = average_stats_by_position.loc[eff_pos]
            proj_by_pos_dict = proj_by_position.to_dict()
            player_projs_dict[player] = proj_by_pos_dict
        


In [56]:
## Add stat projections to salary data
## Need to make sure that marcels names match with names form sportsdata file 
## Or create master file of player id's

df_player_sal['pS/PA'] = df_player_sal.apply(lambda row: round(player_projs_dict[row['PlayerID']]['S'], 3) if row['PlayerID'] in player_projs_dict else np.NaN, axis=1)
df_player_sal['pD/PA'] = df_player_sal.apply(lambda row: round(player_projs_dict[row['PlayerID']]['D'], 3) if row['PlayerID'] in player_projs_dict else np.NaN, axis=1)
df_player_sal['pT/PA'] = df_player_sal.apply(lambda row: round(player_projs_dict[row['PlayerID']]['T'], 3) if row['PlayerID'] in player_projs_dict else np.NaN, axis=1)
df_player_sal['pHR/PA'] = df_player_sal.apply(lambda row: round(player_projs_dict[row['PlayerID']]['HR'], 3) if row['PlayerID'] in player_projs_dict else np.NaN, axis=1)
df_player_sal['pBB/PA'] = df_player_sal.apply(lambda row: round(player_projs_dict[row['PlayerID']]['BB'], 3) if row['PlayerID'] in player_projs_dict else np.NaN, axis=1)
df_player_sal['pHP/PA'] = df_player_sal.apply(lambda row: round(player_projs_dict[row['PlayerID']]['HP'], 3) if row['PlayerID'] in player_projs_dict else np.NaN, axis=1)
df_player_sal['pSB/PA'] = df_player_sal.apply(lambda row: round(player_projs_dict[row['PlayerID']]['SB'], 3) if row['PlayerID'] in player_projs_dict else np.NaN, axis=1)


## Let's add team obp at this step (lead hitters obp - hr) and (trail hitters ops)

In [70]:
df_player_sal.head()

,SlatePlayerID,SlateID,SlateGameID,PlayerID,PlayerGameProjectionStatID,OperatorPlayerID,OperatorSlatePlayerID,OperatorPlayerName,OperatorPosition,OperatorSalary,RemovedByOperator,Team,TeamID,OperatorRosterSlots,Operator,OperatorSlateID,OperatorName,NumberOfGames,OperatorGameType,SalaryCap,EffectivePosition,pS/PA,pD/PA,pT/PA,pHR/PA,pBB/PA,pHP/PA,pSB/PA
0,4255376,9508,58231.0,10000217,2712490.0,390610,12847954,Anthony Rizzo,1B,9800,False,CHC,9.0,[UTIL],DraftKings,27878,NYM vs CHC,1,Showdown,50000.0,1B,0.135,0.046,0.003,0.054,0.115,0.036,0.006
1,4255377,9508,58231.0,10000217,2712490.0,390610,12848092,Anthony Rizzo,1B,14700,False,CHC,9.0,[CPT],DraftKings,27878,NYM vs CHC,1,Showdown,50000.0,1B,0.135,0.046,0.003,0.054,0.115,0.036,0.006
2,4255378,9508,58231.0,10007009,2712470.0,873981,12847951,Pete Alonso,1B,10400,False,NYM,18.0,[UTIL],DraftKings,27878,NYM vs CHC,1,Showdown,50000.0,1B,0.110,0.048,0.004,0.073,0.096,0.025,0.001
3,4255379,9508,58231.0,10007009,2712470.0,873981,12848089,Pete Alonso,1B,15600,False,NYM,18.0,[CPT],DraftKings,27878,NYM vs CHC,1,Showdown,50000.0,1B,0.110,0.048,0.004,0.073,0.096,0.025,0.001
4,4255380,9508,58231.0,10002076,2712508.0,549381,12847958,Kris Bryant,3B/OF,9000,False,CHC,9.0,[UTIL],DraftKings,27878,NYM vs CHC,1,Showdown,50000.0,3B,0.131,0.058,0.005,0.046,0.122,0.030,0.003


In [57]:
## Projected PA
## 1. Batting Order - check
## 2. Team Total - check
## 3. Just Team OBP?

In [58]:
## Get Vegas lines for the day's games

today = "2019-JUN-21"
#today = date.today()
#today = today.strftime('%Y-%b-%d')

response = requests.get(f'https://api.sportsdata.io/api/mlb/fantasy/json/DfsSlatesByDate/{today}', headers={'Ocp-Apim-Subscription-Key': '6fcab751d8594ce9909283dcdc522d24'})
games = response.json()
df_games = pd.json_normalize(games, record_path =['DfsSlateGames'])
df_games = df_games[['SlateGameID', 'GameID', 'OperatorGameID', 'Game.Season', 'Game.Day', 'Game.AwayTeam', 'Game.HomeTeam', 'Game.AwayTeamID', 'Game.HomeTeamID', 'Game.StadiumID', 'Game.AwayTeamProbablePitcherID', 'Game.HomeTeamProbablePitcherID', 'Game.PointSpread', 'Game.OverUnder', 'Game.AwayTeamMoneyLine', 'Game.HomeTeamMoneyLine']]

result_df = df_player_sal.merge(df_games, how='left', on = ['SlateGameID'])


In [59]:
result_df['HomeOrAway'] = result_df.apply(lambda row: 'AWAY' if row['Game.AwayTeamID'] == row['TeamID'] else 'HOME', axis=1)
result_df['PlayerTeamMoneyLine'] = result_df.apply(lambda row: row['Game.AwayTeamMoneyLine'] if row['HomeOrAway'] == 'AWAY' else row['Game.HomeTeamMoneyLine'], axis=1)
result_df['PlayerTeamPointSpread'] = result_df.apply(lambda row: abs(row['Game.PointSpread']) * -1 if row['PlayerTeamMoneyLine'] < 0 else abs(row['Game.PointSpread']), axis=1)
result_df['PlayerTeamVegasWinProb'] = result_df.apply(lambda row: 100 / (100 + row['PlayerTeamMoneyLine']) if row['PlayerTeamMoneyLine'] > 0 else row['PlayerTeamMoneyLine'] / (row['PlayerTeamMoneyLine'] - 100), axis=1)
result_df['PlayerTeamTotal'] = result_df.apply(lambda row: round((row['Game.OverUnder'] / 2) - ((row['PlayerTeamPointSpread'] * (100 / (abs(row['PlayerTeamMoneyLine']) + 100))) / 2), 2), axis=1)


In [60]:
## Getting batting order data from past games
## Need to figure out a way to get this information daily

os.chdir('..')
batting_order = pd.read_csv('battingorder2018-2020.csv')
batting_order.rename(columns = {'sports_data_playerid': 'PlayerID', 'sports_data_game_id': 'GameID'}, inplace = True)
batting_order.drop(['full name'], axis = 1, inplace = True)

merged_df = result_df.merge(batting_order, how = 'left', on = ['PlayerID', 'GameID'])


In [61]:
starters_obp_data = merged_df[merged_df['battingorderposition'].notna()]
starters_obp_data = starters_obp_data[['PlayerID', 'TeamID', 'pS/PA', 'pD/PA', 'pT/PA', 'pHR/PA', 'pBB/PA', 'pHP/PA']].drop_duplicates(['PlayerID'])
starters_obp_data = starters_obp_data[['TeamID', 'pS/PA', 'pD/PA', 'pT/PA', 'pHR/PA', 'pBB/PA', 'pHP/PA']].reset_index(drop=True).groupby(['TeamID']).mean()
starters_obp_data['pOBP'] = starters_obp_data.apply(lambda row: row['pS/PA'] + row['pD/PA'] + row['pT/PA'] + row['pHR/PA'] + row['pBB/PA'] + row['pHP/PA'], axis=1)
merged_df['startersOBP'] = merged_df.apply(lambda row: starters_obp_data.loc[row['TeamID'], 'pOBP'] if row['TeamID'] in list(starters_obp_data.index) else 0, axis=1)

## If not in starting lineup just give a 1 PA projection
merged_df['pPA'] = merged_df.apply(lambda row: 1 if pd.isnull(row['battingorderposition']) else round(3.3 + (-0.12 * row['battingorderposition']) + (.036 * row['PlayerTeamTotal']) + (3.92 * row['startersOBP']), 2), axis=1)


In [62]:
merged_df['OBP-HR'] = merged_df.apply(lambda row: row['pS/PA'] + row['pD/PA'] + row['pT/PA'] + row['pBB/PA'] + row['pHP/PA'], axis=1)
merged_df['pAB/PA'] = merged_df.apply(lambda row: 1 - row['pBB/PA'] - row['pHP/PA'], axis=1)


In [63]:
## Could figure out how to handle pitchers in the 9 spot

temp_df = merged_df[['PlayerID', 'TeamID', 'battingorderposition', 'pS/PA', 'pD/PA', 'pT/PA', 'pBB/PA', 'pHP/PA']].drop_duplicates(['PlayerID']).dropna(subset=['battingorderposition']).reset_index(drop=True)
leadOBP_dict = {}

for index, row in temp_df.iterrows():
    
    teamid = row['TeamID']
    playerid = row['PlayerID']
    b_order = row['battingorderposition']
    lead_hitters = []
    
    if b_order > 3:
        lead_hitters = [b_order - 1, b_order - 2, b_order - 3]
    elif b_order == 3: lead_hitters = [9, 2, 1]
    elif b_order == 2: lead_hitters = [9, 8, 1]
    else:
        lead_hitters = [7, 8, 9]
    
    
    sub_df = temp_df[temp_df['TeamID'] == teamid].reset_index(drop=True)
    boolean_series = sub_df.battingorderposition.isin(lead_hitters)
    filtered_df = sub_df[boolean_series].reset_index(drop=True)
    OB_events = (filtered_df['pS/PA'].sum() + filtered_df['pD/PA'].sum() + filtered_df['pT/PA'].sum() + filtered_df['pBB/PA'].sum() + filtered_df['pHP/PA'].sum()) / filtered_df.shape[0]
    
    #SLG_events = filtered_df.S.sum() + (2 * filtered_df.D.sum()) + (3 * filtered_df['T'].sum()) + (4 * filtered_df.HR.sum())


    leadOBP_dict[playerid] = OB_events

In [64]:
## Could figure out how to handle pitchers in the 9 spot

temp_df = merged_df[['PlayerID', 'TeamID', 'battingorderposition', 'pS/PA', 'pD/PA', 'pT/PA', 'pHR/PA', 'pBB/PA', 'pHP/PA', 'pAB/PA']].drop_duplicates(['PlayerID']).dropna(subset=['battingorderposition']).reset_index(drop=True)
trailOPS_dict = {}

for index, row in temp_df.iterrows():
    
    teamid = row['TeamID']
    playerid = row['PlayerID']
    b_order = row['battingorderposition']
    lead_hitters = []
    
    if b_order < 7:
        lead_hitters = [b_order + 1, b_order + 2, b_order + 3]
    elif b_order == 7: lead_hitters = [9, 8, 1]
    elif b_order == 8: lead_hitters = [9, 2, 1]
    else:
        lead_hitters = [1, 2, 3]
    
    
    sub_df = temp_df[temp_df['TeamID'] == teamid].reset_index(drop=True)
    boolean_series = sub_df.battingorderposition.isin(lead_hitters)
    filtered_df = sub_df[boolean_series].reset_index(drop=True)
    OB_events = (filtered_df['pS/PA'].sum() + filtered_df['pD/PA'].sum() + filtered_df['pT/PA'].sum() + filtered_df['pHR/PA'].sum() + filtered_df['pBB/PA'].sum() + filtered_df['pHP/PA'].sum()) / filtered_df.shape[0]
    SLG_events = (filtered_df['pS/PA'].sum() + (2 * filtered_df['pD/PA'].sum()) + (3 * filtered_df['pT/PA'].sum()) + (4 * filtered_df['pHR/PA'].sum()) / filtered_df['pAB/PA'].mean()) / filtered_df.shape[0]
    
    OPS = OB_events + SLG_events


    trailOPS_dict[playerid] = OPS

In [65]:
merged_df['leadhittersOBP-HR'] = merged_df.apply(lambda row: leadOBP_dict[row['PlayerID']] if row['PlayerID'] in leadOBP_dict else 0, axis=1)
merged_df['trailhittersOPS'] = merged_df.apply(lambda row: trailOPS_dict[row['PlayerID']] if row['PlayerID'] in trailOPS_dict else 0, axis=1)
merged_df['pBA'] = merged_df.apply(lambda row: (row['pS/PA'] + row['pD/PA'] + row['pT/PA'] + row['pHR/PA']) / row['pAB/PA'], axis=1)



In [66]:
merged_df['pRBI/PA'] = merged_df.apply(lambda row: -0.065 + (1.3 * row['pHR/PA']) + (0.3 * row['leadhittersOBP-HR']) + (0.2 * row['pBA']), axis=1)
merged_df['pR-HR/PA'] = merged_df.apply(lambda row: -0.055 + (.245 * row['OBP-HR']) + (0.092 * row['trailhittersOPS']), axis=1)
merged_df['pR/PA'] = merged_df.apply(lambda row: row['pR-HR/PA'] + row['pHR/PA'], axis=1)

In [67]:
merged_df['pS'] = merged_df.apply(lambda row: round(row['pS/PA'] * row['pPA'], 2), axis=1)
merged_df['pD'] = merged_df.apply(lambda row: round(row['pD/PA'] * row['pPA'], 2), axis=1)
merged_df['pT'] = merged_df.apply(lambda row: round(row['pT/PA'] * row['pPA'], 2), axis=1)
merged_df['pHR'] = merged_df.apply(lambda row: round(row['pHR/PA'] * row['pPA'], 2), axis=1)
merged_df['pBB'] = merged_df.apply(lambda row: round(row['pBB/PA'] * row['pPA'], 2), axis=1)
merged_df['pHP'] = merged_df.apply(lambda row: round(row['pHP/PA'] * row['pPA'], 2), axis=1)
merged_df['pR'] = merged_df.apply(lambda row: round(row['pR/PA'] * row['pPA'], 2), axis=1)
merged_df['pRBI'] = merged_df.apply(lambda row: round(row['pRBI/PA'] * row['pPA'], 2), axis=1)
merged_df['pSB'] = merged_df.apply(lambda row: round(row['pSB/PA'] * row['pPA'], 2), axis=1)


In [68]:

merged_df['DraftKingsPoints'] = merged_df.apply(lambda row: round((3 * row['pS']) + (5 * row['pD']) + (8 * row['pT']) + (10 * row['pHR']) + (2 * row['pRBI']) + (2 * row['pR']) + (2 * row['pBB']) + (2 * row['pHP']) + (5 * row['pSB']), 2), axis=1)
merged_df['FanDuelPoints'] = merged_df.apply(lambda row: round((3 * row['pS']) + (6 * row['pD']) + (9 * row['pT']) + (12 * row['pHR']) + (3.5 * row['pRBI']) + (3.2 * row['pR']) + (3 * row['pBB']) + (3 * row['pHP']) + (6 * row['pSB']), 2), axis=1)
                                                
                                                
                                                

In [69]:
projection_df = merged_df[['PlayerID','SlateID', 'Operator', 'OperatorPlayerID', 'TeamID', 'Team', 'OperatorSalary','OperatorGameType', 'SalaryCap', 'OperatorPlayerName', 'OperatorPosition', 'OperatorRosterSlots', 'Team', 'pPA', 'pR', 'pS', 'pD', 'pT', 'pHR', 'pRBI', 'pBB', 'pHP', 'pSB', 'DraftKingsPoints', 'FanDuelPoints']].reset_index(drop=True)
projection_df


,PlayerID,SlateID,Operator,OperatorPlayerID,TeamID,Team,OperatorSalary,OperatorGameType,SalaryCap,OperatorPlayerName,OperatorPosition,OperatorRosterSlots,Team,pPA,pR,pS,pD,pT,pHR,pRBI,pBB,pHP,pSB,DraftKingsPoints,FanDuelPoints
0,10000217,9508,DraftKings,390610,9.0,CHC,9800,Showdown,50000.0,Anthony Rizzo,1B,[UTIL],CHC,4.45,0.70,0.60,0.20,0.01,0.24,0.68,0.51,0.16,0.03,9.53,12.78
1,10000217,9508,DraftKings,390610,9.0,CHC,14700,Showdown,50000.0,Anthony Rizzo,1B,[CPT],CHC,4.45,0.70,0.60,0.20,0.01,0.24,0.68,0.51,0.16,0.03,9.53,12.78
2,10007009,9508,DraftKings,873981,18.0,NYM,10400,Showdown,50000.0,Pete Alonso,1B,[UTIL],NYM,4.48,0.70,0.49,0.22,0.02,0.33,0.77,0.43,0.11,0.00,10.05,13.49
3,10007009,9508,DraftKings,873981,18.0,NYM,15600,Showdown,50000.0,Pete Alonso,1B,[CPT],NYM,4.48,0.70,0.49,0.22,0.02,0.33,0.77,0.43,0.11,0.00,10.05,13.49
4,10002076,9508,DraftKings,549381,9.0,CHC,9000,Showdown,50000.0,Kris Bryant,3B/OF,[UTIL],CHC,4.57,0.71,0.60,0.27,0.02,0.21,0.64,0.56,0.14,0.01,9.56,12.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3701,10005557,9528,FanDuel,101796,13.0,SEA,2000,Classic,35000.0,Braden Bishop,OF,"[OF, UTIL]",SEA,1.00,0.04,0.14,0.04,0.01,0.02,0.01,0.07,0.01,0.01,1.21,1.45
3702,10005513,9528,FanDuel,68538,24.0,OAK,2000,Classic,35000.0,Franklin Barreto,SS,"[SS, UTIL]",OAK,1.00,0.05,0.14,0.04,0.01,0.04,0.03,0.07,0.01,0.01,1.47,1.80
3703,10005569,9528,FanDuel,101832,24.0,OAK,2000,Classic,35000.0,Skye Bolt,OF,"[OF, UTIL]",OAK,1.00,0.04,0.14,0.05,0.01,0.03,0.02,0.09,0.01,0.01,1.42,1.73
3704,10005671,9528,FanDuel,96742,11.0,TB,2000,Classic,35000.0,Nick Ciuffo,C,"[C/1B, UTIL]",TB,1.00,0.05,0.14,0.04,0.01,0.03,0.02,0.08,0.01,0.01,1.37,1.67


In [79]:
cwd = os.getcwd()
os.chdir(cwd + '/Fantasy.2018-2021')


In [81]:
### Evaluation using the optimizer

pitcher_projections = pd.read_csv('june21_pitcher_projections.csv')
pitcher_projections = pitcher_projections[pitcher_projections['SlateID'] == 9439].reset_index(drop=True)
pitcher_projections

,PlayerID,SlateID,Operator,OperatorPlayerID,TeamID,Team,OperatorSalary,OperatorGameType,SalaryCap,OperatorPlayerName,OperatorPosition,OperatorRosterSlots,Team.1,pIP,pW,pQS,pK,pBB,pHR,pH,pHBP,pER,DraftKingsPoints,FanDuelPoints
0,10005311,9439,DraftKings,597956,10.0,CLE,8700,Classic,50000.0,Mike Clevinger,SP,['P'],CLE,1.0,0.0,0.0,1.284,0.393,0.111,0.831,0.045,0.362523,3.331554,5.76443
1,10000432,9439,DraftKings,277705,30.0,HOU,11900,Classic,50000.0,Justin Verlander,SP,['P'],HOU,1.0,0.0,0.0,1.218,0.222,0.141,0.798,0.045,0.363523,3.319954,5.56343
2,10005454,9439,DraftKings,875192,33.0,SD,7700,Classic,50000.0,Logan Allen,SP,['P'],SD,1.0,0.0,0.0,0.918,0.342,0.120,0.891,0.045,0.439857,2.439487,4.43443
3,10006758,9439,DraftKings,657914,2.0,CIN,7600,Classic,50000.0,Lucas Sims,SP,['P'],CIN,1.0,0.0,0.0,1.011,0.375,0.135,0.954,0.045,0.451857,2.543887,4.67743
4,10000970,9439,DraftKings,454389,30.0,HOU,11400,Classic,50000.0,Gerrit Cole,SP,['P'],HOU,1.0,0.0,0.0,1.440,0.282,0.123,0.861,0.045,0.308190,3.800820,6.39543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,10009547,9439,DraftKings,602686,20.0,MIN,4700,Classic,50000.0,Andrew Vasquez,RP,['P'],MIN,1.0,0.0,0.0,0.975,0.360,0.108,0.903,0.045,0.415857,2.583487,4.67743
779,10005725,9439,DraftKings,657960,20.0,MIN,5000,Classic,50000.0,Chase De Jong,RP,['P'],MIN,1.0,0.0,0.0,0.774,0.480,0.147,0.966,0.045,0.556857,1.789687,3.65143
780,10009576,9439,DraftKings,829330,35.0,WSH,7000,Classic,50000.0,James Bourque,SP,['P'],WSH,1.0,0.0,0.0,0.879,0.468,0.123,0.933,0.045,0.494857,2.150687,4.15243
781,10002046,9439,DraftKings,658009,15.0,SF,6200,Classic,50000.0,Ty Blach,SP,['P'],SF,1.0,0.0,0.0,0.612,0.363,0.102,1.050,0.045,0.488857,1.621487,3.36943


In [82]:
slate_projection_df = projection_df[projection_df['SlateID'] == 9439].reset_index(drop=True)
slate_projection_df

,PlayerID,SlateID,Operator,OperatorPlayerID,TeamID,Team,OperatorSalary,OperatorGameType,SalaryCap,OperatorPlayerName,OperatorPosition,OperatorRosterSlots,Team,pPA,pR,pS,pD,pT,pHR,pRBI,pBB,pHP,pSB,DraftKingsPoints,FanDuelPoints
0,10007152,9439,DraftKings,943524,30.0,HOU,4400,Classic,50000.0,Yordan Alvarez,OF,[OF],HOU,4.34,0.63,0.62,0.22,0.02,0.26,0.75,0.45,0.03,0.04,9.64,12.80
1,10000691,9439,DraftKings,548190,32.0,MIL,5900,Classic,50000.0,Christian Yelich,OF,[OF],MIL,4.54,0.79,0.70,0.23,0.03,0.33,0.83,0.56,0.05,0.17,12.10,15.99
2,10003122,9439,DraftKings,737794,1.0,LAD,5700,Classic,50000.0,Cody Bellinger,OF,[OF],LAD,4.33,0.72,0.65,0.21,0.03,0.29,0.77,0.58,0.02,0.12,10.92,14.48
3,10000405,9439,DraftKings,454446,23.0,COL,5600,Classic,50000.0,Charlie Blackmon,OF,[OF],COL,4.64,0.71,0.77,0.25,0.07,0.25,0.75,0.31,0.08,0.05,10.57,13.81
4,10006662,9439,DraftKings,317008,21.0,LAA,4100,Classic,50000.0,Justin Upton,OF,[OF],LAA,4.48,0.61,0.56,0.21,0.01,0.24,0.71,0.46,0.04,0.06,9.15,12.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,10009303,9439,DraftKings,1118534,NaN,None,2300,Classic,50000.0,Jarred Kelenic,OF,[OF],None,1.00,0.05,0.14,0.05,0.01,0.04,0.03,0.09,0.01,0.02,1.61,1.98
884,10009359,9439,DraftKings,915772,NaN,None,2300,Classic,50000.0,Cal Raleigh,C,[C],None,1.00,0.05,0.14,0.04,0.00,0.03,0.03,0.08,0.01,0.00,1.26,1.56
885,10006613,9439,DraftKings,830204,NaN,None,2200,Classic,50000.0,Jomar Reyes,3B,[3B],None,1.00,0.05,0.14,0.05,0.00,0.04,0.03,0.09,0.01,0.01,1.48,1.83
886,10008513,9439,DraftKings,1118204,NaN,None,2100,Classic,50000.0,Julio Rodriguez,OF,[OF],None,1.00,0.05,0.14,0.05,0.01,0.04,0.03,0.09,0.01,0.02,1.61,1.98


In [85]:
hitters_points_and_sal = slate_projection_df.loc[:,['PlayerID','OperatorPlayerName', 'OperatorPosition', 'OperatorSalary','DraftKingsPoints']]
pitchers_points_and_sal = pitcher_projections.loc[:,['PlayerID','OperatorPlayerName', 'OperatorPosition', 'OperatorSalary','DraftKingsPoints']]
points_and_sal = pd.concat([hitters_points_and_sal, pitchers_points_and_sal], axis=0)
points_and_sal

,PlayerID,OperatorPlayerName,OperatorPosition,OperatorSalary,DraftKingsPoints
0,10007152,Yordan Alvarez,OF,4400,9.640000
1,10000691,Christian Yelich,OF,5900,12.100000
2,10003122,Cody Bellinger,OF,5700,10.920000
3,10000405,Charlie Blackmon,OF,5600,10.570000
4,10006662,Justin Upton,OF,4100,9.150000
...,...,...,...,...,...
778,10009547,Andrew Vasquez,RP,4700,2.583487
779,10005725,Chase De Jong,RP,5000,1.789687
780,10009576,James Bourque,SP,7000,2.150687
781,10002046,Ty Blach,SP,6200,1.621487


In [86]:
#points_and_sal = slate_projection_df.loc[:,['PlayerID','OperatorPlayerName', 'OperatorPosition', 'OperatorSalary','DraftKingsPoints']]

points_and_sal['OperatorPlayerName'] = points_and_sal['OperatorPlayerName'].str.replace(' ','')

points_and_sal2 = points_and_sal.copy()
points_and_sal2['Position'] = points_and_sal2['OperatorPosition'].str.split('/')

points_and_sal2 = points_and_sal2.explode('Position')

In [113]:
sportsdata_projs = pd.read_csv('PlayerGameProjection.2019.csv')

In [108]:
import pandas as pd
import requests
from pulp import *
import numpy as np
from collections import Counter

In [109]:
players_include =[]
include = points_and_sal2[points_and_sal2['OperatorPlayerName'].isin(players_include)]
points_and_sal2 = points_and_sal2[~points_and_sal2['OperatorPlayerName'].isin(players_include)]


#optimize


points_and_sal3 = points_and_sal2.reset_index().reset_index()

budget = 50000 - include['OperatorSalary'].sum()

dup = points_and_sal3[points_and_sal3['PlayerID'].duplicated(keep=False)]['level_0'].tolist()

players = points_and_sal3['OperatorPlayerName'].tolist()

positions = points_and_sal3['Position'].tolist()

costs = points_and_sal3['OperatorSalary'].tolist()

values = points_and_sal3['DraftKingsPoints'].tolist()

pos_include = include['Position'].tolist()

x = generate_optimal_lineup2(players, positions, values, costs, budget, dup, pos_include)

In [110]:
lineup = points_and_sal3[points_and_sal3['level_0'].isin(x)]

lineup

,level_0,index,PlayerID,OperatorPlayerName,OperatorPosition,OperatorSalary,DraftKingsPoints,Position
1,1,1,10000691,ChristianYelich,OF,5900,12.100000,OF
12,12,12,10000077,FreddieFreeman,1B,4900,10.220000,1B
18,18,18,10000857,GarySánchez,C,5200,10.410000,C
26,26,25,10006969,RonaldAcuñaJr.,OF,4500,10.570000,OF
27,27,26,10001911,FranciscoLindor,SS,4600,9.760000,SS
30,30,28,10005589,AlexBregman,3B/SS,4700,10.710000,3B
135,135,115,10000485,JoseAltuve,2B,3800,9.340000,2B
138,138,118,10000646,JocPederson,OF,4100,10.380000,OF
1122,1122,65,10005232,AaronNola,SP,8000,20.181453,SP
1807,1807,750,10000792,SeanGilmartin,RP,4000,13.772165,RP


In [111]:
lineup['OperatorSalary'].sum()

49700

In [112]:
lineup['DraftKingsPoints'].sum()

117.44361840498684

In [105]:
def generate_optimal_lineup2(players, positions, values, costs, budget, dup, pos_include):
    num_variables = len(players)
    
    lp = LpProblem("My LP Problem", LpMaximize)
    
    d = {}
    
    for i in range(0, num_variables):
        d[i] = LpVariable(str(i), cat="Binary")
    objective = sum(np.array(values) * np.array(list(d.values())))
    lp += objective
    
    #for name in players:
      #  prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if n == name]) <=1
    
    p_constraint = 0
    c_constraint = 0
    oneb_constraint = 0
    twob_constraint = 0
    threeb_constraint = 0
    ss_constraint = 0
    
    of_constraint = 0
    
    for i in range(0, len(positions)):
        if positions[i] == "SP" or positions[i] == "RP":
            p_constraint += d[i]
        elif positions[i] == "C":
            c_constraint += d[i]
        elif positions[i] == "1B":
            oneb_constraint += d[i]
        elif positions[i] == "2B":
            twob_constraint += d[i]
        elif positions[i] == "3B":
            threeb_constraint += d[i]
        elif positions[i] == "SS":
            ss_constraint += d[i]
        elif positions[i] == "OF":
            of_constraint += d[i]
        
    count = Counter(pos_include)
    
    lp += p_constraint == 2 - count['RP'] - count['SP']
    lp += c_constraint == 1 - count['C']
    lp += oneb_constraint == 1 - count['1B']
    lp += twob_constraint == 1 - count['2B']
    lp += threeb_constraint == 1 - count['3B']
    lp += ss_constraint == 1 - count['SS']
    lp += of_constraint == 3 - count['OF']
    
    
    
    for i in range(len(dup)):
        if i%2==0:
            lp += pulp.lpSum([d[dup[i]],d[dup[i+1]]])<=1
        
   
    cost = sum(np.array(costs) * np.array(list(d.values())))
    
    lp += cost <= budget
    
    lp.solve()
    
    lineup = [variable.name for variable in lp.variables() if variable.varValue == 1]
    
    #print(lineup)
    return lineup

In [72]:

date = "2019-JUN-21"

#game slates and salaries

response = requests.get(f'https://api.sportsdata.io/api/mlb/fantasy/json/DfsSlatesByDate/{date}', headers={'Ocp-Apim-Subscription-Key': '6fcab751d8594ce9909283dcdc522d24'})
games = response.json()

#player points

response = requests.get(f'https://api.sportsdata.io/api/mlb/fantasy/json/PlayerGameStatsByDate/{date}', headers={'Ocp-Apim-Subscription-Key': '6fcab751d8594ce9909283dcdc522d24'})
players2 = response.json()

response = requests.get(f'https://api.sportsdata.io/api/mlb/fantasy/json/PlayerGameProjectionStatsByDate/{date}', headers={'Ocp-Apim-Subscription-Key': '6fcab751d8594ce9909283dcdc522d24'})
projections = response.json()

In [73]:
#format json

#game slates
df = pd.json_normalize(games)

#flatten
df_games = pd.json_normalize(games, record_path =['DfsSlateGames'])
df_player_sal = pd.json_normalize(games, record_path =['DfsSlatePlayers'])

#player points
player_points = pd.json_normalize(players2)

#projections
proj = pd.json_normalize(projections)

In [81]:
classic_slate_ids = list(df.loc[(df['OperatorGameType'] == 'Classic') & (df['Operator'] == 'DraftKings')]['SlateID'])

In [83]:
date

'2019-JUN-21'

In [84]:
response = requests.get(f'https://api.sportsdata.io/api/mlb/fantasy/json/PlayerGameStatsByDate/{date}', headers={'Ocp-Apim-Subscription-Key': '6fcab751d8594ce9909283dcdc522d24'})
players2 = response.json()

In [85]:
player_points = pd.json_normalize(players2)

In [86]:
player_points.head()

,StatID,TeamID,PlayerID,SeasonType,Season,Name,Team,Position,PositionCategory,Started,InjuryStatus,GameID,OpponentID,Opponent,Day,DateTime,HomeOrAway,Games,FantasyPoints,AtBats,Runs,Hits,Singles,Doubles,Triples,HomeRuns,RunsBattedIn,BattingAverage,Outs,Strikeouts,Walks,HitByPitch,Sacrifices,SacrificeFlies,GroundIntoDoublePlay,StolenBases,CaughtStealing,OnBasePercentage,SluggingPercentage,OnBasePlusSlugging,Wins,Losses,Saves,InningsPitchedDecimal,TotalOutsPitched,InningsPitchedFull,InningsPitchedOuts,EarnedRunAverage,PitchingHits,PitchingRuns,PitchingEarnedRuns,PitchingWalks,PitchingStrikeouts,PitchingHomeRuns,PitchesThrown,PitchesThrownStrikes,WalksHitsPerInningsPitched,PitchingBattingAverageAgainst,FantasyPointsFanDuel,FantasyPointsDraftKings,WeightedOnBasePercentage,PitchingCompleteGames,PitchingShutOuts,PitchingOnBasePercentage,PitchingSluggingPercentage,PitchingOnBasePlusSlugging,PitchingStrikeoutsPerNineInnings,PitchingWalksPerNineInnings,PitchingWeightedOnBasePercentage
0,2730536,9,10001954,1,2019,Addison Russell,CHC,2B,IF,1,None,55283,18,NYM,2019-06-21T00:00:00,2019-06-21T14:20:00,HOME,1,7.0,3.0,1.0,1.0,0.0,0.0,0.0,1.0,2.0,0.333,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333,1.333,1.666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,22.2,16.0,0.667,0.0,0.0,0.00,0.000,0.000,0.0,0.0,0.000
1,2730535,9,10001234,1,2019,Yu Darvish,CHC,P,P,1,None,55283,18,NYM,2019-06-21T00:00:00,2019-06-21T14:20:00,HOME,1,6.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000,1.000,2.000,0.0,0.0,0.0,6.0,18.0,6.0,0.0,6.0,4.0,4.0,4.0,2.0,6.0,2.0,88.0,58.0,1.0,0.182,24.0,13.9,1.000,0.0,0.0,0.25,0.455,0.705,9.0,3.0,0.333
2,2730531,9,10000213,1,2019,Javier Baez,CHC,SS,IF,1,None,55283,18,NYM,2019-06-21T00:00:00,2019-06-21T14:20:00,HOME,1,4.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,3.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.250,0.000,0.250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,12.2,9.0,0.250,0.0,0.0,0.00,0.000,0.000,0.0,0.0,0.000
3,2730533,9,10007641,1,2019,David Bote,CHC,3B,IF,1,None,55283,18,NYM,2019-06-21T00:00:00,2019-06-21T14:20:00,HOME,1,4.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.250,0.000,0.250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,12.2,9.0,0.500,0.0,0.0,0.00,0.000,0.000,0.0,0.0,0.000
4,2730529,9,10002076,1,2019,Kris Bryant,CHC,RF,OF,1,None,55283,18,NYM,2019-06-21T00:00:00,2019-06-21T14:20:00,HOME,1,2.0,4.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.250,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250,0.500,0.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,6.0,5.0,0.250,0.0,0.0,0.00,0.000,0.000,0.0,0.0,0.000


In [87]:
slate_list

NameError: name 'slate_list' is not defined